In [1]:

import numpy as np
import scipy as sp
import h5py
import os

import SimulatorData as sim_data
import Simulator as sim_system


In [2]:

##! keys values are the columns of interest of exp data,
##! but the values are the ones used by the simulator 
##* the experimental data is read as it is presented, modfications are added later in the Simaulator object
##* gas species must be consistent with the chemical equations from the json file


schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "O mean":       "O",
    "N":            "N",
    "CO mean":      "CO",
    "gamma.1":      "gamma_exp"
}


###* Input files and function

reactions_file = "reactionsSimple.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.6, 'Vdb_S':0.1, 'Odb_S': 0.1}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO2' :  lambda const_dict, exp_dict: compute_remaining_flux(const_dict, exp_dict, 0.032),
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'O', 0.016),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

input_folder_path = "Experimental_data_Paper"

output_folder_path = "Buffer_Data"
exp_data_file = input_folder_path + ".hdf5"

In [3]:

data_loader = sim_data.DataLoader(schema, folder_path=input_folder_path, output_file=exp_data_file, output_folder=output_folder_path)
data_loader.load_data(force_update=True)

Summary Buffer:
File Data : [b'Experimental_data_Paper/gammas_TD2019.xlsx']
col CO: shape(66,)
col File_Data: shape(1,)
col N: shape(66,)
col O: shape(66,)
col Tnw: shape(66,)
col Tw: shape(66,)
col current: shape(66,)
col gamma_exp: shape(66,)
col pressure: shape(66,)


In [4]:


exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)
print(sim.output_parser['rates_model'])
print(sim.output_parser['reactions_list'])



dO2_F/dt = -1.0*O2_F*O_F*r_15 - 1.0*O2_F*r_10 - 1.0*O2_F*r_12 - 1.0*O2_F*r_14 + 1.0*r_9*(-O2_F - O_F + 1.0)
dO_F/dt = -1.0*O2_F*O_F*r_15 - 2.0*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - 1.0*O_F*r_11 - 1.0*O_F*r_2 - 1.0*O_F*r_4 - 0.02*O_F*r_5*(-O_S - Odb_S - Vdb_S + 1.0) + 1.0*r_1*(-O2_F - O_F + 1.0)
dO_S/dt = -1.0*O_F*O_S*r_7 + 1.0*O_F*r_5*(-O_S - Odb_S - Vdb_S + 1.0) - 1.0*O_S*r_16 - 1.0*O_S*r_17 - 1.0*O_S*r_6 + 1.0*r_3*(-O_S - Odb_S - Vdb_S + 1.0)
dOdb_S/dt = -1.0*O_F*Odb_S*r_27 + 1.0*O_F*Vdb_S*r_26 - 1.0*Odb_S*r_23 - 1.0*Odb_S*r_24 - 1.0*Odb_S*r_25 + 1.0*Vdb_S*r_20
dVdb_S/dt = 1.0*O_F*Odb_S*r_27 - 1.0*O_F*Vdb_S*r_26 + 1.0*O_S*r_16 + 1.0*O_S*r_17 + 1.0*Odb_S*r_25 - 1.0*Vdb_S*r_20 - 1.0*Vdb_S*r_21 - 1.0*Vdb_S*r_22 + 1.0*r_18*(-O_S - Odb_S - Vdb_S + 1.0) + 1.0*r_19*(-O_S - Odb_S - Vdb_S + 1.0)
['r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'r_8', 'r_9', 'r_10', 'r_11', 'r_12', 'r_14', 'r_15', 'r_16', 'r_17', 'r_18', 'r_19', 'r_20', 'r_21', 'r_22', 'r

In [5]:


# sim.compute_rates(flag_print=False)
# frac_solutions_arr, gammas_result_arr = sim.solve_system()

# gammas_sum = np.array([sum(gamma_dict.values()) for gamma_dict in gammas_result_arr])

# # print(sim.output_parser['species_model'])
# # print(frac_solutions_arr[:5])
# # print(gammas_result_arr[:5])

# print(gammas_sum)

In [6]:

print(sim.output_parser['reactions_list'])


reactions_list = sim.output_parser["reactions_list"]

E_1 = 0.0
nu_D = 2e13

dict_mod_vec = [
    {"id": None, "rate": "adsorption", "model_dict": {"E": E_1}},
    {"id": None, "rate": None, "model_dict": {"nu_D": nu_D}}
]

id2index_model = {r["id"]:idx for idx, r in enumerate(reactions_list)}

# print(id2index_model[1])

print(reactions_list)

for dict_mod in dict_mod_vec:
    mod_update = dict_mod["model_dict"]
    if dict_mod["id"] is not None:
        idx = id2index_model[dict_mod["id"]] 
        reactions_list[idx]['model_dict'].update(mod_update)
        
    else:
        if dict_mod["rate"] is not None:
            idx2update = [idx for idx, reaction in enumerate(reactions_list) if reaction["rate"] == dict_mod["rate"]]
            for idx in idx2update:
                reactions_list[idx]["model_dict"].update(mod_update)
        else:
            
            if len(mod_update) == 1:
                param_update = next(iter(mod_update))
            else:
                print("Dictionaire has multiple keys")
            
            idx2update = []
            for idx, reaction in enumerate(reactions_list):
                reaction_model_dict = reactions_list[idx]['model_dict']
                if param_update in reaction_model_dict.keys():
                    idx2update.append(idx)
            
            for idx in idx2update:
                reactions_list[idx]["model_dict"].update(mod_update)
            
        
        

print("after update")

print(reactions_list)



for reaction in reactions_list:
    print(reaction["id"], reaction["model_dict"])





[{'id': 1, 'left': {'V_F': 1}, 'right': {'O_F': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': False}, {'id': 2, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'desorption', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 30.0, 'nu_d': 1000000000000000.0}, 'gamma': False}, {'id': 3, 'left': {'V_S': 1}, 'right': {'O_S': 1}, 'rate': 'adsorption', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': False}, {'id': 4, 'left': {'O_F': 1}, 'right': {'V_F': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 0.0}, 'gamma': True}, {'id': 5, 'left': {'O_F': 1, 'V_S': 1}, 'right': {'V_F': 1, 'O_S': 1}, 'rate': 'diffusion', 'gas_specie': '', 'model_dict': {'SF': 1.0, 'E': 15.0, 'nu_D': 10000000000000.0, 'factor': 0.75}, 'gamma': False}, {'id': 6, 'left': {'O_S': 1}, 'right': {'V_S': 1}, 'rate': 'recomb_ER', 'gas_specie': 'O', 'model_dict': {'SF': 1.0, 'E': 15.0}, 'gamma': True}, {'id': 7, 'left': {'O_F': 1, 'O_S': 1

In [7]:


sim.compute_rates(flag_print=False)
frac_solutions_arr, gammas_result_arr = sim.solve_system()

gammas_sum = np.array([sum(gamma_dict.values()) for gamma_dict in gammas_result_arr])

print(gammas_sum)
# print(sim.output_parser['species_model'])
# print(frac_solutions_arr[:5])
# print(gammas_result_arr[:5])

[0.00184266 0.00238171 0.00270187 0.00428002 0.00509798 0.00559299
 0.00216837 0.00277705 0.00304263 0.00390811 0.00501654 0.0058705
 0.0068684  0.00769896 0.00775239 0.00072984 0.00071733 0.00074109
 0.00085978 0.0011048  0.00129485 0.00153851 0.00164406 0.00095443
 0.00086027 0.00084832 0.00100065 0.00127394 0.00149911 0.00178114
 0.0020419  0.00213779 0.00052649 0.00043489 0.00038499 0.00042223
 0.00050594 0.00056896 0.00064056 0.00070573 0.00071611 0.00053392
 0.00043102 0.00046508 0.00055815 0.00063504 0.00074937 0.00089615
 0.00092734 0.00049586 0.00040246 0.00034508 0.00035733 0.00039648
 0.00042815 0.00047429 0.00052811 0.00065829 0.00048364 0.00037426
 0.00038842 0.00043687 0.00048028 0.00055558 0.00067826 0.00077113]
